<a href="https://colab.research.google.com/github/JHyunjun/TF2.0_2D-CNN_Reg/blob/main/2D_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

tf.random.set_seed(1)

#Data Load
df_train = pd.read_csv('/dir')
df_test = pd.read_csv('/dir')
train_data_input = df_train.iloc[:,:-1]
train_data_output = df_train.iloc[:,-1]
test_data_input = df_test.iloc[:,:-1]
test_data_output = df_test.iloc[:,-1]
print(train_data_input.shape,train_data_input)
print(train_data_output.shape, train_data_output)

In [ ]:
#Normalization
def MinMaxScaler(data):
  denom = np.max(data,0) - np.min(data,0)
  nume = data-np.min(data,0)
  return nume/denom

def back_MinMax(data,max,min):
  diff = max-min
  back = data * diff + min
  return back



In [ ]:
#Hyper parameters
seqLength = 10
dataDim = df_test.shape[1]
outputDim = 1
hiddenDim = 10
lr = 0.001
iterations = 10
batch_size = 32

train_data = df_train.iloc[:,:]
test_data = df_test.iloc[:,:]
print(train_data)
print(test_data)

trainSize = int(len(train_data))
trainSet = train_data[0:trainSize]
testSize = int(len(test_data))
testSet = test_data[0:testSize]
testSet_for_result = testSet

trainSet_max = np.max(trainSet)
trainSet_min = np.min(trainSet)
testSet_max = np.max(testSet)
testSet_min = np.min(testSet)
trainSet_max = (trainSet_max[dataDim-1])
trainSet_min = (trainSet_min[dataDim-1])
testSet_max = (testSet_max[dataDim-1])
testSet_min = (testSet_min[dataDim-1])


trainSet = MinMaxScaler(trainSet)
testSet = MinMaxScaler(testSet)

print(trainSet.shape) # (5807,5)
print(testSet.shape) # (10882,5)

In [ ]:
#Data Windowing
def sliding_window(time_series, seq_length):
    dataX = []
    dataY = []
    for i in range(0, len(time_series) - seq_length): 
        print(i)
        _x = time_series.iloc[i:i + seq_length, : -1 ]
        _y = time_series.iloc[i + seq_length, [-1]]  
        print(_x, "->", _y)
        dataX.append(_x)
        dataY.append(_y)
    return np.array(dataX), np.array(dataY)

trainX, trainY = sliding_window(trainSet, seqLength)
testX, testY = sliding_window(testSet, seqLength)

In [ ]:
# Hyper Parameter
filters = 100
kernel_size = (2,2)
pooling_size = (2,2)
kernel_size2 = (2,2)
epochs = 100
stride = (1,1)
input_shape = train_data_input.shape[1]
input_rank = train_data_input.shape[0]

#2D-CNN Model
model = Sequential()
model.add(Conv2D(filters = filters, kernel_size = kernel_size, strides = stride, activation = 'relu', input_shape = (seqLength,input_shape,1)))
model.add(Conv2D(filters = filters, kernel_size = kernel_size2, strides = stride, activation = 'relu'))
model.add(MaxPooling2D(pool_size = pooling_size))
model.add(Flatten())
model.add(Dense(48, activation = 'relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.summary()

In [ ]:
model.fit(trainX,trainY, epochs = epochs, verbose = 0)

In [ ]:
# Evaluate Testset
res2 = model.evaluate(trainX, trainY, batch_size = batch_size)
print("Error Rate : ",res2)

xhat = testX
yhat = model.predict(xhat)

#Estimating
estimated_testY = back_MinMax(yhat,testSet_max,testSet_min)

real = testSet_for_result.iloc[seqLength:,-1] # estimated + seqLength
real = real.to_numpy()

print("real.shape : ",real.shape) 
print("real : ",real)

print("estimated_testY.shape : ",estimated_testY.shape)
print("estimated_testY",estimated_testY)

plt.figure()
plt.plot(estimated_testY[:len(estimated_testY)],label = "Estimated by 2D-CNN")
plt.plot(real[:len(real)], label = "Real")
plt.legend(prop = {'size':10})

print("1132")